In [1]:
!python --version

Python 3.10.12


In [2]:
!git clone https://github.com/THU-MIG/yolov10.git

Cloning into 'yolov10'...
remote: Enumerating objects: 20329, done.
remote: Counting objects: 100% (1527/1527), done.
remote: Compressing objects: 100% (176/176), done.
remote: Total 20329 (delta 1450), reused 1363 (delta 1351), pack-reused 18802
Receiving objects: 100% (20329/20329), 11.19 MiB | 12.60 MiB/s, done.
Resolving deltas: 100% (14326/14326), done.


In [3]:
# # Google Colab Specific commands
from google.colab import drive
drive.mount('/content/drive')
# # # Example: Unzip into the current working directory - since i have a zip in my drive and i want to unzip into my colab env for faster access
!unzip -q "/content/drive/MyDrive/Colab Notebooks/pravin_thesis/0_dataset/10_balanced_data_split_roi_label.zip" -d '/content/yolov10/datasets/'

Mounted at /content/drive


In [4]:
cd yolov10

/content/yolov10


In [5]:
!pip install .

Processing /content/yolov10
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.meta

In [6]:
import os
import urllib.request

# Create a directory for the weights in the current working directory
weights_dir = os.path.join(os.getcwd(), "weights")
os.makedirs(weights_dir, exist_ok=True)

# URLs of the weight files
# Trying Yolov10-S
urls = [
    # "https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10n.pt",
    "https://github.com/jameslahm/yolov10/releases/download/v1.0/yolov10s.pt"
]

# Download each file
for url in urls:
    file_name = os.path.join(weights_dir, os.path.basename(url))
    urllib.request.urlretrieve(url, file_name)
    print(f"Downloaded {file_name}")

Downloaded /content/yolov10/weights/yolov10s.pt


## Training

In [ ]:
!yolo task=detect mode=train epochs=100 batch=16 plots=True model=weights/yolov10s.pt data=./datasets/data.yaml patience=20 cache=true single_cls=true

# Try to reduce the learning rate a little, it is getting penelized a lot

New https://pypi.org/project/ultralytics/8.2.75 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.34 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=weights/yolov10s.pt, data=./datasets/data.yaml, epochs=100, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, val_period=1, cache=True, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show

In [ ]:
# Google Colab specific code
# Copy run results to google drive
!cp -r ./runs "../drive/MyDrive/Colab Notebooks/pravin_thesis/1_yolov10_object_detection/3_results_Yolov10-S_roi_only"


## Prediction

In [7]:
# Load trained weight
!cp '/content/drive/MyDrive/Colab Notebooks/pravin_thesis/1_yolov10_object_detection/3_results_Yolov10-S_roi_only/runs/detect/train/weights/best.pt' '/content/'

In [8]:
!yolo task=detect mode=predict conf=0.25 save=True model=../best.pt source=./datasets/dataset/valid/images/cocci.1001.jpg

Ultralytics YOLOv8.1.34 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv10s summary (fused): 293 layers, 8035734 parameters, 0 gradients, 24.4 GFLOPs

image 1/1 /content/yolov10/datasets/dataset/valid/images/cocci.1001.jpg: 640x640 2 ROIs, 16.8ms
Speed: 5.1ms preprocess, 16.8ms inference, 369.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
# TODO: Evaluation of the test dataset
# Make change in data.yaml file: https://stackoverflow.com/questions/78073911/yolov8-how-to-calculate-map-on-test-set